In [2]:
year = 2023
day = 19

In [73]:
from aocd import submit
from aocd.models import Puzzle
from functools import reduce
import numpy as np

puzzle = Puzzle(year=year, day=day)
data = puzzle.input_data
# data = puzzle.examples[0].input_data

data = data.strip()
all_rules, ranges = data.split("\n\n")
all_rules = all_rules.split("\n")
ranges = ranges.split("\n")


all_rules = [d.split("{") for d in all_rules]
all_rules = {d[0]: d[1].replace("}", "").split(",") for d in all_rules}


ranges = [d.replace("{", "").replace("}", "").split(",") for d in ranges]
ranges = [[int(n.split("=")[1]) for n in d] for d in ranges]
all_rules

{'dc': ['m<2420:R', 'R'],
 'jb': ['s>2618:vv', 'R'],
 'vxv': ['m>2362:tsk', 'm>1949:hpf', 'qdx'],
 'xs': ['a>2562:R', 'a>2291:R', 's<3703:R', 'A'],
 'tl': ['x<1583:hb', 'nd'],
 'xk': ['m<3191:R', 'a>844:R', 'a<714:A', 'A'],
 'sxh': ['m>128:trk', 'a<521:qvc', 'zq'],
 'cxv': ['s>1082:A', 's<1006:R', 'R'],
 'jsm': ['a>776:A', 'm<2436:R', 'A'],
 'sn': ['a<295:A', 'x<1774:fpm', 'fsk'],
 'lld': ['m<2279:A', 'R'],
 'ng': ['a<590:sdl', 'a>609:snx', 'R'],
 'sdl': ['s<1719:A', 'a<553:A', 's>1738:A', 'A'],
 'hrd': ['s>3215:A', 'a>691:A', 'm>2571:R', 'R'],
 'jq': ['s<2384:A', 's<2587:zlc', 'txr'],
 'gp': ['x>3310:R', 's<3337:R', 'R'],
 'nk': ['a<3026:jbp', 'sm'],
 'kth': ['s>2224:A', 'R'],
 'plr': ['x<1171:A', 'x>1291:A', 'R'],
 'mzl': ['m>2788:R', 'R'],
 'clc': ['s>2151:R', 'a<1437:mr', 's>1993:gj', 'A'],
 'tzk': ['x<3188:A', 'A'],
 'df': ['s<127:A', 'R'],
 'cjz': ['x<551:rr', 'a<3279:kpg', 'm<3044:R', 'R'],
 'zpn': ['s<1044:R', 'a<928:R', 'x>667:A', 'A'],
 'qlq': ['a<3396:A', 'm<188:A', 'R'],
 '

In [74]:
from collections import defaultdict
cache = defaultdict(list)
cache["in"] = ranges

In [75]:
def apply_rule(rule, part):
    x, m, a, s = part
    for r in rule:
        if ":" in r:
            if eval(r.split(":")[0]):
                return r.split(":")[1]
        else:
            return r

In [76]:
out = defaultdict(list)
while cache:
    new_cache = defaultdict(list)
    for key, ranges in cache.items():
        new_parts = []
        for part in ranges:
            new_rule = apply_rule(all_rules[key], part)
            if new_rule in "AR":
                out[new_rule].append(part)
            else:
                new_cache[new_rule].append(part)
    cache = new_cache

In [77]:
answer = sum(sum(xmas) for xmas in out["A"])
submit(answer, part="a", year=year, day=day)

Part a already solved with same answer: 386787


In [78]:
# idea
# start with ranges
# start with in rule
# split ranges based on rules
# keep track of possible ranges for each rule
# when a rule is resolved, add to the count

cache = defaultdict(list)
cache["in"] = [[range(1, 4001), range(1, 4001), range(1, 4001), range(1, 4001)]]

out = defaultdict(list)
c_order = {"x": 0, "m": 1, "a": 2, "s": 3}
for i in range(100):
    new_cache = defaultdict(list)
    print(dict(cache))
    for key, ranges in cache.items():
        new_parts = []
        for part_range in ranges:
            for r in all_rules[key]:
                # print(r)
                # print(part_range)
                if ":" in r:
                    c, op, val, new_key = r[0], r[1], int(r.split(":")[0][2:]), r.split(":")[1]
                    if op == ">":
                        val += 1
                        new_part_range = part_range.copy()
                        new_part_range[c_order[c]] = range(val, part_range[c_order[c]].stop)

                        part_range[c_order[c]] = range(part_range[c_order[c]].start, val)
                    elif op == "<":
                        new_part_range = part_range.copy()
                        new_part_range[c_order[c]] = range(part_range[c_order[c]].start, val)
                        part_range[c_order[c]] = range(val, part_range[c_order[c]].stop)
                    else:
                        raise ValueError

                    if new_key in ['A', 'R']:
                        out[new_key].append(new_part_range)
                    else:
                        new_cache[new_key].append(new_part_range)

                elif r in ['A', 'R']:
                    out[r].append(part_range)
                else:
                    new_cache[r].append(part_range)
    if not new_cache:
        print(f"finished in {i} iterations")
        break
    cache = new_cache

{'in': [[range(1, 4001), range(1, 4001), range(1, 4001), range(1, 4001)]]}
{'vh': [[range(1, 4001), range(1, 4001), range(1, 4001), range(1749, 4001)]], 'th': [[range(1, 4001), range(1, 4001), range(1, 4001), range(1, 1749)]]}
{'fcr': [[range(1, 4001), range(2349, 4001), range(1, 4001), range(1749, 4001)]], 'dp': [[range(1, 4001), range(1340, 2349), range(1, 4001), range(1749, 4001)]], 'rrl': [[range(1, 4001), range(1, 1340), range(1, 4001), range(1749, 4001)]], 'vlx': [[range(1, 4001), range(1, 4001), range(2283, 4001), range(1, 1749)]], 'pbt': [[range(1, 4001), range(1, 4001), range(1080, 2283), range(1, 1749)]], 'bfz': [[range(1, 4001), range(1, 4001), range(362, 1080), range(1, 1749)]], 'fhl': [[range(1, 4001), range(1, 4001), range(1, 362), range(1, 1749)]]}
{'bvh': [[range(1993, 4001), range(2349, 4001), range(1, 4001), range(1749, 4001)]], 'kk': [[range(1, 1197), range(2349, 4001), range(1, 4001), range(1749, 4001)]], 'sz': [[range(1197, 1993), range(2349, 4001), range(1, 4001),

In [79]:
import math
sum(math.prod(len(r) for r in xmas_r) for xmas_r in out["A"])

131029523269531

In [80]:
len(range(1,4001))

4000

In [ ]:
all_char_rules = {}
for c in "xmas":
    char_rules = [1, 4001]
    for _, key in all_rules.items():
        for r in key:
            if c in r and (">" in r or "<" in r):
                print(r)
                split_val = int(r.split(":")[0][2:])
                if r[1] == ">":
                    split_val += 1
                char_rules.append(split_val)
                char_rules.sort()
    all_char_rules[c] = char_rules

from itertools import pairwise, product
ranges_dict = {}
for c, char_rules in all_char_rules.items():
    r = []
    for a, b in pairwise(char_rules):
        r.append(range(a,b))
    ranges_dict[c] = r


def resolve_rule(part):
    rule = "in"
    while not rule in "AR":
        rule = apply_rule(all_rules[rule], part)
    return rule

out = defaultdict(int)
i = 0
for x, m, a, s in product(*ranges_dict.values()):
    if i % 10_000_000 == 0:
        print(i)
    n_values = len(x) * len(m) * len(a) * len(s)
    outcome = resolve_rule((x.stop-1, m.stop-1, a.stop-1, s.stop-1))
    out[outcome] += n_values
    i += 1

out